### Obtaining the best perfoming hyperparameter for NN model of the environment
This notebook compares the raw feature selection from the state against the feature construction based on domain knowlegde

In [1]:
# the required imports
import numpy as np
import pandas as pd
# from linear_aproximation import Model
# from environment import network
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# additional import for the Neural Network
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# path = '/Users/mawongh/OneDrive/REFERENCE FILE/D/Disertation/brainstorming/'
path = '/home/mawongh/ws/datasets/'
dataset = pd.read_pickle(path + 'full_dataset.pickle')
dataset.tail(3)

,action,reward,s_next,state
299475,121.0,-2.915486,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."
299476,121.0,-3.054551,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."
299477,121.0,-2.653020,"[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250...","[2733, 2733, 2733, 3200, 3200, 3200, 2500, 250..."


In [3]:
np.random.seed(1973)
N = len(dataset)
sample_size = 75000
sample_indexes = np.random.choice(np.arange(N), size = sample_size, replace=False)
study_dataset = dataset.iloc[sample_indexes]

S = np.array([x for x in study_dataset.state])
a_r = study_dataset[['action', 'reward']].values


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


S = scaler.fit_transform(S)  # Train the transformer object so it knows what means and variances to use

/home/mawongh/anaconda3/envs/rl/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [ ]:
n_models = 8

from sklearn.model_selection import KFold

kf = KFold(n_splits=n_models, random_state=42)

S_train = []
S_eval = []
a_train = []
a_eval = []
r_train = []
r_eval = []

for train_index, eval_index in kf.split(S):
    S_train.append(S[train_index])
    S_eval.append(S[eval_index])
    
    a_r_train, a_r_eval = a_r[train_index], a_r[eval_index]
    
    a_train.append(a_r_train[:,0])
    a_eval.append(a_r_eval[:,0])
    
    r_train.append(a_r_train[:,1])
    r_eval.append(a_r_eval[:,1])
# end for


In [ ]:
from keras import optimizers

models = [Sequential() for i in np.arange(n_models)]
n_cols = S.shape[1]

# model 1
models[0].add(Dense(200, activation = 'relu', input_shape=(n_cols,)))
models[0].add(Dense(200, activation = 'relu'))
models[0].add(Dense(126, activation = 'linear'))
models[0].compile(optimizer=optimizers.Adam(), loss='mean_squared_error')

# model 2
models[1].add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
models[1].add(Dense(400, activation = 'relu'))
models[1].add(Dense(126, activation = 'linear'))
models[1].compile(optimizer=optimizers.Adam(), loss='mean_squared_error')

# model 3
models[2].add(Dense(200, activation = 'relu', input_shape=(n_cols,)))
models[2].add(Dense(200, activation = 'relu'))
models[2].add(Dense(126, activation = 'linear'))
models[2].compile(optimizer=optimizers.RMSprop(), loss='mean_squared_error')

# model 4
models[3].add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
models[3].add(Dense(400, activation = 'relu'))
models[3].add(Dense(126, activation = 'linear'))
models[3].compile(optimizer=optimizers.RMSprop(), loss='mean_squared_error')

# model 5
models[4].add(Dense(200, activation = 'relu', input_shape=(n_cols,)))
models[4].add(Dense(200, activation = 'relu'))
models[4].add(Dense(200, activation = 'relu'))
models[4].add(Dense(126, activation = 'linear'))
models[4].compile(optimizer=optimizers.Adam(), loss='mean_squared_error')

# model 6
models[5].add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
models[5].add(Dense(400, activation = 'relu'))
models[5].add(Dense(400, activation = 'relu'))
models[5].add(Dense(126, activation = 'linear'))
models[5].compile(optimizer=optimizers.Adam(), loss='mean_squared_error')

# model 7
models[6].add(Dense(200, activation = 'relu', input_shape=(n_cols,)))
models[6].add(Dense(200, activation = 'relu'))
models[6].add(Dense(200, activation = 'relu'))
models[6].add(Dense(126, activation = 'linear'))
models[6].compile(optimizer=optimizers.RMSprop(), loss='mean_squared_error')

# model 8
models[7].add(Dense(400, activation = 'relu', input_shape=(n_cols,)))
models[7].add(Dense(400, activation = 'relu'))
models[7].add(Dense(400, activation = 'relu'))
models[7].add(Dense(126, activation = 'linear'))
models[7].compile(optimizer=optimizers.RMSprop(), loss='mean_squared_error')

In [ ]:
def train_model(model_number, num_epochs = 20):
    for epoch in np.arange(num_epochs):
        print('model: {}, epoch: {}'.format(model_number + 1, epoch))
        for step in np.arange(len(S_train[model_number])):
            s = S_train[model_number][step].reshape(1,-1)
            r = models[model_number].predict_on_batch(s)
            r[0,int(a_train[model_number][step])] = r_train[model_number][step]
            models[model_number].train_on_batch(s,r)

for m in np.arange(n_models):
    train_model(model_number = m, num_epochs = 20)

In [ ]:

for m in np.arange(n_models):
    y_pred = models[m].predict(S_eval[m])

    r_hat = np.array([y_pred[i,int(a_eval[m][i])] for i in np.arange(len(S_eval[m]))])
    
    eval_MSE = mean_squared_error(r_eval[m], r_hat)
    print('model: {}, eval MSE: {}'.format(m,eval_MSE))


In [ ]:

for m in np.arange(n_models):
    y_pred = models[m].predict(S_train[m])

    r_hat = np.array([y_pred[i,int(a_train[m][i])] for i in np.arange(len(S_train[m]))])
    
    train_MSE = mean_squared_error(r_train[m], r_hat)
    print('model: {}, train MSE: {}'.format(m,train_MSE))

In [ ]:
x = np.arange(len(S_test))
plt.plot(x, r_test)
plt.plot(x, r_hat)
plt.show()

train_MSE = mean_squared_error(r_train, r_train_hat)
print('train set MSE: {}'.format(train_MSE))


test_MSE = mean_squared_error(r_test, r_hat)
print('test set MSE: {}'.format(test_MSE))

In [ ]:
n_cols = S_train.shape[1]
model = Sequential()
# input layer
model.add(Dense(200, activation = 'relu', input_shape=(n_cols,)))
# second layer hidden
model.add(Dense(200, activation = 'relu'))
# output layer
model.add(Dense(126, activation = 'linear'))
# compiles the model
model.compile(optimizer='sgd', loss='mean_squared_error')

# print(model)

# train the neural network
# model.fit(Xraw_train, yraw_train, epochs = 10)

# # instantiate linear regression models and fits them
# reg = linear_model.LinearRegression()
# print('fitting model 1...')
# reg.fit(X_train, y_train)

In [ ]:
s = S_train[0].reshape(1, -1)
q = model.predict_on_batch(s)
q[0,int(A_test[0])] = y_test[0]
model.train_on_batch(s, q)

In [ ]:
for j in np.arange(30):
    print(j)
    for i in np.arange(len(S_train)):
        s = S_train[i].reshape(1,-1)
        q = model.predict_on_batch(s)
        q[0,int(A_train[i])] = y_train[i]
        model.train_on_batch(s,q)


In [ ]:
# yhat = []
# for i in np.arange(len(S_test)):
#     s = S_test[i].reshape(1,-1)
#     q = model.predict_on_batch(s)
#     yhat += [q[0,int(A_test[i])]]
    

ypred = model.predict(S_test)

yhat = np.array([ypred[i,int(A_test[i])] for i in np.arange(len(S_test))])

# yhat = np.array(yhat)

# y_hat = model.predict(Xraw_test)


# # Do the prediction and calculate the performance (MSE) for model 1
# # Xtest_transformed = scaler.transform(X_test)
# # x = np.arange(len(Xtest_transformed))
x = np.arange(len(S_test))
# y_hat = reg.predict(X_test)
plt.plot(x, y_test)
plt.plot(x, yhat)
plt.show()

# y_train_hat = model.predict(Xraw_train)
# train_MSE = mean_squared_error(yraw_train, y_train_hat)
# print('train set MSE: {}'.format(train_MSE))

test_MSE = mean_squared_error(y_test, yhat)
print('test set MSE: {}'.format(test_MSE))




# regraw = linear_model.LinearRegression()
# print('fitting model 2...')
# regraw.fit(Xraw_train, yraw_train)

In [ ]:
# S_test.shape
# prueba = S_test[0].reshape(1,-1).tolist()
# prueba
ypred = model.predict(S_test)
np.max(np.max(ypred, axis=1))

In [ ]:
# model.predict_on_batch(prueba)

## Playground

In [ ]:
np.array([[1,2], [1], [2,3]])

In [ ]:
np.random.choice(20, 30, replace=False)

In [ ]:
np.min([20, 30])

In [ ]:
100 % 100

In [ ]:
model.get_weights()

In [ ]:
s = np.array([10, 20, 40, 50])
s.reshape(1,-1)[0]

In [ ]:
model.get_weights()s.reshape(-1,1)

In [ ]:
model.save('test.h5')




In [ ]:
print("%03d" % (100,))